In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tqdm.notebook as tqdm

### Scraping Boliga website for data

In [35]:
import requests
from bs4 import BeautifulSoup
import re
import json

def houses_boliga(number_houses):
    """
    Returns a list of all ids for houses on boliga
    """
    house_id = list()
    url = "https://www.boliga.dk/resultat?"
    
    for i in range(int(number_houses/50)):
        new_url = url + f"?page={i}"
        response = requests.get(new_url)
        html = response.text
        soup = BeautifulSoup(html,"html.parser")
        ids = soup.find_all("a",{"class":"house-list-item"})
        link_houses = list()

        for link in ids:
            link_houses.append(re.findall("(/\d{4,}/)",link["href"])[0].replace("/",""))
        
        house_id.extend(link_houses)
        
    return house_id

def get_info(id_list):
    
    """
    Take an list with ids of houses on boliga and gets specific data about these ids
    
    """
    all_df = list()
    new_keys = ["registeredArea","downPayment","estateUrl","currentArchiveId","forSaleNowId",
                "foreclosureId","selfsaleEstateId","cleanStreet","estateId","latitude","longitude",
               "propertyType","priceChangePercentTotal","energyClass","price","rooms","size","lotSize",
               "floor","buildYear","city","isActive","municipality","zipCode","street",
                "squaremeterPrice","daysForSale","createdDate","basementSize","views"]
    
    for house_id in id_list:
        response = requests.get(f'https://api.boliga.dk/api/v2/estate/{house_id}')
        response = response.json()
        df_dict = {key: response[key] for key in new_keys}
        df = pd.DataFrame(df_dict,index=[0])
        all_df.append(df)

    df = pd.concat(all_df,axis=0,ignore_index=True)
    
    return df

def get_reviews(df):
    bodys = list()
    i=0
    for link in tqdm.tqdm(df["estateUrl"].values[100:110]):
        i += 1
        response = requests.get(link)
        html = response.text
        soup = BeautifulSoup(html,"html.parser")
        
        if link[8:15] =="home.dk": #Home
            ids = soup.find_all("div",{"class":"text"},"p")
            bodys.extend([x.p.text.replace("\n","").strip().lower() for x in ids if len(x)>1])
        elif link[8:15] =="ww.skbo": #skbolig
            ids = soup.find_all("div",{"class":"listing-text"})
            bodys.extend([sk.text.replace("\n","").replace("\r","").strip().lower() for sk in ids if len(sk)>1])
        elif link[8:15] == "www.nyb": #Nybolig
            ids = soup.find_all("div",{"class":"foldable-spot__container"})
            bodys.extend([ny.text.replace("\n","").strip().lower() for ny in ids if len(ny)>1])
        elif link [8:15] == "ww.elto": #Eltoft Nielsen
            ids = soup.find_all("br")
            bodys.extend([elto.text.replace("\n","").strip().lower() for elto in ids if len(elto)>1])
        elif link[8:15] == "www.cla": #Claus Borg
            ids = soup.find_all("div",{"id":"case_content"})
            bodys.extend([cla.text.replace("\n","").strip().lower() for cla in ids if len(cla)>1])
        elif link[8:15] == "www.lok": #Lokalbolig
            ids = soup.find_all("p")
            loka = [lok.text.replace("\n","").strip().lower() for lok in ids if len(lok.text)>100]
            bodys.extend([''.join(loka)])
        elif link[8:15] == "www.edc": #EDC Bolig
            ids = soup.find_all("div",{"class":"description"})
            bodys.extend([edc.text.replace("\n","").strip().lower() for edc in ids if len(edc)>1])
        elif link[8:15] == "adamsch": #Adam Schnack
            ids = soup.find_all("div",{"class":"listing-text"})
            bodys.extend([adam.text.replace("\n","").strip().lower() for adam in ids if len(adam)>1])
        elif link[8:15] == "www.est": #Estate
            ids = soup.find_all("div",{"class":"property-description"})
            bodys.extend([est.text.replace("\n","").strip().lower() for est in ids if len(est)>1])
        elif link[8:15] == "www.bri": #Brikk Ejendomme
            ids = soup.find_all("div",{"class":"prop-user-content"})
            bodys.extend([bri.text.replace("\n","").strip().lower() for bri in ids if len(bri)>1])
        elif link[8:15] == "www.rea": #Realmæglerne
            ids = soup.find_all("div",{"class":"text-full"})
            bodys.extend([rea.text.replace("\n","").strip().lower() for rea in ids if len(rea)>1])
        elif link[8:15] == "danboli": #Danbolig
            ids = soup.find_all("div",{"class":"db-description-block"})
            bodys.extend([dan.text.replace("\n","").strip().lower() for dan in ids if len(dan)>1])
        elif link[8:15] == "ww.lili": #Lillenhof
            ids = soup.find_all("div",{"class":"inner"})
            bodys.extend([dan.text.replace("\n","").strip().lower() for dan in ids if len(dan)>10])
        elif link[8:15] == "lbaeks.": #elbæks
            bodys.append(np.nan)
        elif link[8:15] == "ww.paul":
            bodys.append(np.nan)
        elif link[8:15] == "bjornby":
            ids = soup.find_all("div",{"class":"content d-md-block d-none wrap-content"})
            bodys.extend([bjor.text.replace("\n","").strip() for bjor in ids if len(bjor)>10])
        else:
            print(link,"not in loop")
            
            
        if len(bodys) != i:
            bodys.append(np.nan)
        
    
    return bodys

def preprocess_csv(csv):
    """
    This function loads the dataset from boliga annd preproccesses it.
    """
    df = pd.read_csv(csv)
    y = np.array(df["price"])
    
    
    
    return df


def preprocess_text():
    
    
    return None


In [39]:
df = read_boliga("house_data.csv")
df.head()

,Unnamed: 0,registeredArea,downPayment,estateUrl,currentArchiveId,forSaleNowId,foreclosureId,selfsaleEstateId,cleanStreet,estateId,...,city,isActive,municipality,zipCode,street,squaremeterPrice,daysForSale,createdDate,basementSize,views
0,0,68,225000,https://home.dk/boligkatalog/koebenhavn/1051/e...,1676546,0,0,0,Nyhavn,0,...,København K,True,101,1051,"Nyhavn 31B, st..",60472.0,56,2020-06-24T23:55:52.010Z,0,401
1,1,78,430000,https://www.nybolig.dk/ejerlejlighed/1051/nyha...,1688236,0,0,0,Nyhavn,0,...,København K,True,101,1051,Nyhavn 23 4,51777.0,22,2020-07-28T23:40:31.220Z,0,206
2,2,117,460000,http://www.skbolig.dk/sag.asp?sagsnr=221920&mg...,1672807,0,0,0,Nyhavn,0,...,København K,True,101,1051,"Nyhavn 12D, st.",73200.0,65,2020-06-16T03:15:59.387Z,0,108
3,3,146,550000,http://www.skbolig.dk/sag.asp?sagsnr=333420&mg...,1694633,0,0,0,Nyhavn,0,...,København K,True,101,1051,"Nyhavn 51, 1+2.",62471.0,1,2020-08-18T22:02:51.587Z,0,269
4,4,182,625000,http://www.skbolig.dk/sag.asp?sagsnr=331320&mg...,1689738,0,0,0,Nyhavn,0,...,København K,True,101,1051,"Nyhavn 12, 3.",64766.0,16,2020-08-03T22:02:34.863Z,0,105


In [21]:
np.unique(df["city"].values)

array(['København K'], dtype=object)

In [5]:
bodys

['det berømte nyhavn er en af københavns allerstørste turistattraktioner, hvor historiske, farvestrålende og spændende huse kan opleves, hyggelige restauranter besøges og de fascinerende både betragtes. på trods af den livlige atmosfære, gemmer der sig bag glasporten i nyhavn 31 en skøn og rolig baggård, hvor lige præcis denne drømmelejlighed er placeret. \xa0en lille trappe sørger for, at i får jeres helt egen private indgang til den højtbeliggende stuelejlighed. i træder ind i det store leverum, hvor køkken og stue ligger i en pragtfuld, lys og åben kombination. hvide vægge, gulve og loft samt blottede bjælker pryder indretningen, og tilfører et moderne, rå udtryk, som inviterer til sociale sammenkomster, harmoniske aftener foran brændeovnen samt lækker madlavning i det flotte køkken.\xa0det store, indretningsvenlige soveværelse kan prale med et stort indbygget skab, som i også finder i entréen, så i ikke behøver at bekymre jer om opbevaringsplads. det sidste stop på turen er det ele

In [6]:
from nltk.Corpora import wordnet as wn
"dan" in wn.langs()

ModuleNotFoundError: No module named 'nltk.Corpora'

In [36]:
df_id = pd.read_csv("/Users/frederikfindsen/Documents/økonomi/Sommerskole - ISD/SODAS2020_group2/id_houses.csv")
len(np.unique(df_id["ids"].values))

52104

In [38]:

df_id


,Unnamed: 0,ids
0,0,1676546
1,1,1688236
2,2,1672807
3,3,1694633
4,4,1689738
...,...,...
52099,52099,1664830
52100,52100,1645428
52101,52101,1664119
52102,52102,1479257


In [11]:
from nltk.corpus import wordnet as wn
wn.synsets("hund", lang="dan")

[Synset('dog.n.01')]

In [12]:
synonyms = []
antonyms = []
for syn in wordnet.synsets("hund",lang="dan"):
    for l in syn.lemmas():
        synonyms.append(l.name())
        if l.antonyms():
            antonyms.append(l.antonyms()[0].name())

In [13]:
synonyms

['dog', 'domestic_dog', 'Canis_familiaris']